# Process scl data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from scipy.signal import find_peaks, find_peaks_cwt
import pandas as pd
import sys
sys.path.append('..')

In [3]:
from cybb_mist.scl_processing import process_gsr_pipeline, low_pass_filter
from cybb_mist.trigger_mapping import parse_trigger_series
from cybb_mist.path_conf import interim_data_dir, processed_data_dir

from plotly_resampler import FigureWidgetResampler
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from tqdm.auto import tqdm
import os

### Test the extraction on a single file

In [4]:
# First, process the aligned EDF files
for pqt in tqdm(list(interim_data_dir.rglob("*/SCL_*.parquet"))):
    print(pqt)
    condition = pqt.name.split('.')[0].split('_')[1]

    # Read & process the ECG file
    df_scl = pd.read_parquet(pqt)

    df_trigger = parse_trigger_series(df_scl["trigger"])
    start, end = df_trigger.t_start.iloc[0], df_trigger.t_start.iloc[-1]
    df_scl_proc = process_gsr_pipeline(df_scl["SCL"][start:end].rename('EDA'))

    break


  0%|          | 0/124 [00:00<?, ?it/s]

/users/jonvdrdo/jonas/data/cyberball/interim/4/SCL_mist.parquet


In [23]:
df_scl_proc['EDA_Phasic'].dropna().index.to_series().diff().value_counts()

0 days 00:00:00.100000    37439
Name: timestamp, dtype: int64

In [25]:
df_scl_proc['SCR_Peaks_scipy_reduced'].value_counts()

1.0    83
Name: SCR_Peaks_scipy_reduced, dtype: int64

In [26]:
df_scl_proc['EDA_lf_cleaned_tonic_lf'].dropna().index.to_series().diff().value_counts()

0 days 00:00:00.100000    37439
Name: timestamp, dtype: int64

In [27]:
df_scl_proc['EDA_SQI_smoothend'].dropna().index.to_series().diff().value_counts()

0 days 00:00:00.100000    38393
Name: timestamp, dtype: int64

## Apply the extraction on all the files

In [32]:
# First, process the aligned EDF files
for pqt in tqdm(list(interim_data_dir.rglob("*/SCL_*.parquet"))):
    try:
        print(pqt)
        condition = pqt.name.split(".")[0].split("_")[1]

        # Read & process the ECG file
        df_scl = pd.read_parquet(pqt)

        df_trigger = parse_trigger_series(df_scl["trigger"])
        start, end = df_trigger.t_start.iloc[0], df_trigger.t_start.iloc[-1]
        df_scl_proc = process_gsr_pipeline(df_scl["SCL"][start:end].rename("EDA"))
        df_scl_proc = df_scl_proc[
            [
                "EDA_lf_cleaned_tonic_lf",
                "EDA_SQI_smoothend",
                "EDA_Phasic",
                "SCR_Peaks_scipy_reduced",
            ]
        ]

        # Save the processed file in feat stat dir
        scl_stat_dir = processed_data_dir.joinpath(pqt.parent.name)
        if not scl_stat_dir.exists():
            os.makedirs(scl_stat_dir)
        df_scl_proc.to_parquet(
            scl_stat_dir.joinpath(f"df_scl_proc_{condition}.parquet")
        )
    except Exception as e:
        print(e)


  0%|          | 0/124 [00:00<?, ?it/s]

/users/jonvdrdo/jonas/data/cyberball/interim/4/SCL_mist.parquet
/users/jonvdrdo/jonas/data/cyberball/interim/4/SCL_cybb.parquet
/users/jonvdrdo/jonas/data/cyberball/interim/45/SCL_mist.parquet
/users/jonvdrdo/jonas/data/cyberball/interim/45/SCL_cybb.parquet
/users/jonvdrdo/jonas/data/cyberball/interim/63/SCL_cybb.parquet
/users/jonvdrdo/jonas/data/cyberball/interim/17/SCL_mist.parquet
/users/jonvdrdo/jonas/data/cyberball/interim/17/SCL_cybb.parquet
/users/jonvdrdo/jonas/data/cyberball/interim/54/SCL_mist.parquet
/users/jonvdrdo/jonas/data/cyberball/interim/54/SCL_cybb.parquet
/users/jonvdrdo/jonas/data/cyberball/interim/69/SCL_cybb.parquet
/users/jonvdrdo/jonas/data/cyberball/interim/34/SCL_mist.parquet
/users/jonvdrdo/jonas/data/cyberball/interim/34/SCL_cybb.parquet
/users/jonvdrdo/jonas/data/cyberball/interim/53/SCL_mist.parquet
/users/jonvdrdo/jonas/data/cyberball/interim/53/SCL_cybb.parquet
/users/jonvdrdo/jonas/data/cyberball/interim/22/SCL_mist.parquet
/users/jonvdrdo/jonas/data/

## Playground

In [ ]:
USER = 73
PARADIGM = "cybb"

df_scl = pd.read_parquet(
    list(interim_data_dir.joinpath(str(USER)).glob(f"SCL_{PARADIGM}*"))[0]
)


In [ ]:
out = process_gsr_pipeline(df_scl['SCL'].rename('EDA'))
out.columns

In [ ]:
import numpy as np

In [ ]:
def delta_sqi(
    eda_series: pd.Series,
    noise_series: pd.Series,
    noise_threshold=0.1,
    min_delta_threshold=0.02,
    max_increase=0.25,  # the ratio which a signal can increase in a second
    max_decrease=0.1,  # the ratio which a signal can decrease in a second
    fs=10,
    window_s: int = 1,
    output_name: str = "eda_delta_SQI",
) -> pd.Series:
    eda_med = eda_series.rolling(fs * window_s, center=True).median()
    increase_threshold = np.maximum(
        min_delta_threshold, eda_med * max_increase / fs
    )
    decrease_threshold = (
        np.maximum(min_delta_threshold, eda_med * max_decrease / fs) * -1
    )
    delta = eda_series.diff()
    valid_decrease = (
        # and slow decrease | harsher decrease but rather low noise
        (delta >= decrease_threshold)
        | (delta >= 2 * decrease_threshold) & (noise_series <= noise_threshold / 2)
    )
    # we will more severly focus on decrease masks
    decrease_mask = valid_decrease.rolling(2 * fs).min().shift(-2*fs + 1).astype(bool)
    valid_delta = (
        # either slow increase | harsher increase but rather low noise!
        ((delta <= 0.3 * increase_threshold) & ~decrease_mask)
        | (delta <= increase_threshold) & decrease_mask
        | (delta <= 2 * increase_threshold) & (noise_series <= noise_threshold / 2)
    ) & valid_decrease
    return valid_delta.rename(output_name)

In [ ]:
delta_sqi(out['EDA_lf_2Hz'], out['noise_mean_2s']).astype(int).plot()

In [ ]:
# find_peaks(out["EDA_Phasic"], height=0.015, distance=10, prominence=0.03)

In [ ]:
s_ff = low_pass_filter(
    out["EDA_Phasic"], f_cutoff=2, fs=10, output_name="EDA_Phasic_2Hz"
)
s_fp, s_fp_d = find_peaks(
    s_ff,
    height=0.01,
    distance=10 * 1,
    prominence=0.15,
    wlen=10 * 180,
)

s_fp_d['peak_index'] = s_fp
df_peak = pd.DataFrame(s_fp_d)
df_peak['left_bases'] = (df_peak['peak_index'] - df_peak['left_bases']) / 10
df_peak['right_bases'] = (df_peak['right_bases'] - df_peak['peak_index']) / 10
df_peak = df_peak.rename(columns={'left_bases': 'rise_time_s', 'right_bases': 'fall_time_s'})
'rise_time: ' + df_peak['rise_time_s'].astype(str) + '\n' + 'fall_time: ' + df_peak.fall_time_s.astype(str)

In [ ]:
df_peak

In [ ]:
(df_peak['rise_time_s'] < 14).value_counts()

In [ ]:
import plotly_resampler

In [ ]:
fig = FigureWidgetResampler(make_subplots(rows=3, cols=1, shared_xaxes=True))

for col in ["EDA", "EDA_lf_2Hz"]:
    fig.add_trace(go.Scatter(name=col), hf_x=out[col].index, hf_y=out[col])

for col in ["SCR_Peaks_neurokit_reduced", "SCR_Peaks_neurokit"]:
    s = out[out[col] == 1]["EDA"]
    fig.add_trace(
        go.Scatter(
            name=col,
            mode="markers",
            marker_symbol="cross",
            marker_size=10,
        ),
        limit_to_view=True,
        hf_x=s.index,
        hf_y=s,
    )

for col in ["EDA_Phasic"]:
    fig.add_trace(
        go.Scatter(name=col), hf_x=out[col].index, hf_y=out[col], row=2, col=1
    )

# s_ff: pd.Series = low_pass_filter(
#     out["EDA_Phasic"], f_cutoff=1, fs=10, output_name="EDA_Phasic_1Hz"
# )
# fig.add_trace(go.Scatter(name=s_ff.name), hf_x=s_ff.index, hf_y=s_ff, row=2, col=1)
s_ff: pd.Series = low_pass_filter(
    out["EDA_Phasic"], f_cutoff=2, fs=10, output_name="EDA_Phasic_2Hz"
)
fig.add_trace(go.Scatter(name=s_ff.name), hf_x=s_ff.index, hf_y=s_ff, row=2, col=1)
# s_ff: pd.Series = low_pass_filter(
#     out["EDA_Phasic"], f_cutoff=4, fs=10, output_name="EDA_Phasic_4Hz"
# )
# fig.add_trace(go.Scatter(name=s_ff.name), hf_x=s_ff.index, hf_y=s_ff, row=2, col=1)


s_ff = low_pass_filter(
    out["EDA_Phasic"], f_cutoff=2, fs=10, output_name="EDA_Phasic_2Hz"
)
s_fp = s_ff.iloc[
    find_peaks(
        s_ff,
        height=0.01,
        distance=10 * 1,
        prominence=0.1,
        wlen=10 * 120,
    )[0]
]
fig.add_trace(
    go.Scatter(
        name="scipy find peaks",
        mode="markers",
        marker_symbol="cross",
        marker_size=10,
    ),
    limit_to_view=True,
    hf_x=s_fp.index,
    hf_y=s_fp,
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(name="noise"),
    hf_x=out["noise_area_1s"].index,
    hf_y=out["EDA_Phasic"] / out["noise_area_1s"],
    row=3,
    col=1,
)

del s_fp, s, col, s_ff

fig.update_layout(height=900)
fig
